In [ ]:
from pathlib import Path
import os
import sys
import warnings
from IPython.display import display, HTML, Markdown
            
from msticpy.nbtools import nbinit
nbinit.init_notebook(namespace=globals())
)

from msticpy.data_proc.sql_to_kql import sql_to_kql

In [ ]:
sql_to_kql

In [ ]:
{'from': {'value': {...}}, 'select': [{...}, {...}]}